##### Mamy co prawda dodatkowe benefity dla naszych najlepszych klientów, ale może dałobysię ustalić kto potencjalnie jest skłonny wydawać u nas więcej?

### Etap 1

1. Definicja problemu biznesowego, zdefiniowanie zadania/zadań modelowania i wszystkich założeń, zaproponowanie kryteriów sukcesu.
2. Analiza danych z perspektywy realizacji tych zadań (trzeba ocenić, czy dostarczonedane są wystarczające – może czegoś brakuje, może coś trzeba poprawić, domagać się innych danych).

#### Kontekst
- sklep internetowy z elektroniką i grami komputerowymi.

#### Zadanie biznesowe
- identyfikacja klientów, którzy często oglądają różne produkty, ale później ich nie kupują.

#### Biznesowe kryterium sukcesu
- przynajmniej 30% ze zidentyfikowanych klientów, którym sklep przypomni o oglądanych wcześniej produktach (i zaoferuje np. zniżkę), zdecyduje się na zakup.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os, glob

inputs = []

for each in glob.glob(os.path.join(os.getcwd(),'**/*.jsonl'), recursive=True):
    print(f'Name with path: {each} \nName without path: {os.path.basename(each)}')
    inputs.append(each)

Name with path: /home/bindas/Documents/IUM/IUM-project/data/deliveries.jsonl 
Name without path: deliveries.jsonl
Name with path: /home/bindas/Documents/IUM/IUM-project/data/sessions.jsonl 
Name without path: sessions.jsonl
Name with path: /home/bindas/Documents/IUM/IUM-project/data/products.jsonl 
Name without path: products.jsonl
Name with path: /home/bindas/Documents/IUM/IUM-project/data/users.jsonl 
Name without path: users.jsonl


In [3]:
users = pd.read_json(inputs[0], lines=True)
deliveries = pd.read_json(inputs[1], lines=True)
sessions = pd.read_json(inputs[2], lines=True)
products = pd.read_json(inputs[3], lines=True)

In [4]:
users.describe()

,purchase_id,delivery_company
count,5297.000000,5045.000000
mean,22649.000000,498.261249
std,1529.256519,106.646521
min,20001.000000,360.000000
25%,21325.000000,360.000000
50%,22649.000000,516.000000
75%,23973.000000,620.000000
max,25297.000000,620.000000


In [5]:
users.sample(3)

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
2386,22387,2020-04-24T16:31:40,None,360.0
4190,24191,2021-02-01T18:35:13,2021-02-04T18:35:21,516.0
584,20585,2021-02-28T13:08:04,2021-03-04T13:10:24,516.0


In [6]:
deliveries.describe()

,session_id,user_id,product_id,offered_discount,purchase_id
count,37539.000000,35657.000000,35696.000000,37539.000000,5297.000000
mean,105310.580729,199.224388,1128.044851,4.921815,22649.000000
std,3049.208276,57.835868,119.183838,7.074449,1529.256519
min,100001.000000,102.000000,1001.000000,0.000000,20001.000000
25%,102677.000000,150.000000,1035.000000,0.000000,21325.000000
50%,105326.000000,196.000000,1067.000000,0.000000,22649.000000
75%,107956.000000,252.000000,1278.000000,10.000000,23973.000000
max,110597.000000,301.000000,1319.000000,20.000000,25297.000000


In [7]:
deliveries.sample(3)

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
32238,109098,2021-02-23 01:44:36,273.0,1283.0,VIEW_PRODUCT,0,NaN
17972,105089,2021-02-19 19:22:33,193.0,1281.0,VIEW_PRODUCT,0,NaN
20724,105850,2020-09-19 10:22:38,205.0,1053.0,VIEW_PRODUCT,5,NaN


In [12]:
deliveries.event_type.unique()

array(['VIEW_PRODUCT', 'BUY_PRODUCT'], dtype=object)

In [8]:
sessions.describe()

,product_id,price
count,319.000000,3.190000e+02
mean,1160.000000,5.370644e+06
std,92.231593,4.231655e+07
min,1001.000000,-2.450000e+02
25%,1080.500000,2.390000e+01
50%,1160.000000,4.499000e+01
75%,1239.500000,1.109900e+02
max,1319.000000,6.887800e+08


In [9]:
sessions.sample(3)

,product_id,product_name,category_path,price
45,1046,LEGO Batman 2 DC Super Heroes (PS3),Gry i konsole;Gry na konsole;Gry PlayStation3,61.18
194,1195,Titan Quest (PC),Gry i konsole;Gry komputerowe,38.90
268,1269,Space Hack (PC),Gry i konsole;Gry komputerowe,19.99


In [10]:
products.describe()

,user_id
count,200.000000
mean,201.500000
std,57.879185
min,102.000000
25%,151.750000
50%,201.500000
75%,251.250000
max,301.000000


In [11]:
products.sample(3)

,user_id,name,city,street
172,274,Olgierd Sobków,Szczecin,ul. Zielona 01/53
77,179,Nikodem Zioła,Szczecin,plac Perłowa 225
67,169,Anastazja Sobkiewicz,Radom,aleja Kolorowa 281
